## Machine Learning: Initialize Trained Network

In [1]:
#PyTorchの初期化
import torch
import torchvision

#ランダム関数
import random

### import randoms
import math

### import colors
from colorama import Fore, Back, Style

model = torchvision.models.alexnet(pretrained=False)

#種類を増やした場合、変更するところ
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 3)

#モデルの読み込み
model.load_state_dict(torch.load('best_model.pth'))

#カメラのフォーマットを学習済みモデルのフォーマットに合わせ変換
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

#カメラ画像の取得
from jetcam.csi_camera import CSICamera
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224)
image = camera.read()
image_widget = ipywidgets.Image(format='jpeg')
image_widget.value = bgr8_to_jpeg(image)

#カメラ画像の更新
camera.running = True

def update_image(change):
    image = change['new']
    image_widget.value = bgr8_to_jpeg(image)
    
camera.observe(update_image, names='value')

#GPU
device = torch.device('cuda')
model = model.to(device)


## Machine Learning: Deduction using the trained network

In [2]:
#推論
import torch.nn.functional as F
import time
import sys
a=0
one_blocked=0.0
two_blocked=0.0
three_blocked=0.0
def update(change):
    global blocked_slider, robot,a
    global one_blocked, two_blocked, three_blocked
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    one_blocked = float(y.flatten()[0])
    two_blocked = float(y.flatten()[1])
    three_blocked = float(y.flatten()[2])
    if(one_blocked > two_blocked and one_blocked > three_blocked):
        a = 0
    elif(two_blocked > one_blocked and two_blocked > three_blocked):
        a = 1
    elif(three_blocked > one_blocked and three_blocked > one_blocked):
        a = 2
    #0　グー rock 
    #1　パー paper
    #2　チョキ scissors   
    
    time.sleep(0.001)
        
update({'new': image})  # we call the function once to intialize


## Gameplay: actual gameplay process

In [3]:
### Functions

def name_of_value(val):
    if val == 0:
        return "Rock    ";
    if val == 1:
        return "Paper   ";
    if val == 2:
        return "Scissor ";


### MAIN

# activate hand gesture recognition
display(image_widget)
camera.observe(update, names='value')

# main process
game_count = won_count = 0
TIME_DELTA = 0.7
try:
    while True: # forever loop
        # wait for signal
        sys.stdout.write("\n\rAre you ready?") 
        time.sleep(2.0)
        #GPIO.wait_for_edge(BUTTON_PIN, GPIO.RISING)
        
        # reset light and rotation
        game_count = game_count+1
        sys.stdout.flush()
        sys.stdout.write("\rGame %2s: Rock-" % game_count)
        time.sleep( TIME_DELTA )    
        
        # Rock-
        sys.stdout.flush()        
        sys.stdout.write("\rGame %2s: Paper-" % game_count)
        time.sleep( TIME_DELTA )    
        
        # Paper-
        sys.stdout.flush()
        sys.stdout.write("\rGame %2s: Scissors-" % game_count)
        time.sleep( TIME_DELTA )
              
        # Scissors (GO!)
        sys.stdout.flush()
        sys.stdout.write("\rGame %2s: GO!" % game_count )
        
        rint = random.randint(0,2)
              
        # Wait a little and detect hand gesture
        time.sleep( TIME_DELTA )
        sint = a; 
        sys.stdout.write("\rGame %2s  " % game_count)
        sys.stdout.write("Opponent: %s You: %s " % (name_of_value(rint), name_of_value(sint)))  
        sys.stdout.write("\tR:%f, P:%f, S:%f\t" % (one_blocked, two_blocked, three_blocked) )
        
        # win or lose
        diff = 0
        diff = (rint - sint) % 3;
        if diff == 0:
            sys.stdout.write(Fore.YELLOW+"Draw")
            #tie_game();
        elif diff == 1:
            sys.stdout.write(Fore.RED+"You lose")
            #won_game();
        elif diff == 2:
            sys.stdout.write(Fore.GREEN + "You win!!")
            won_count = won_count + 1
            #lost_game();
        sys.stdout.write(Style.RESET_ALL)
        
except KeyboardInterrupt:
    sys.stdout.write(Style.RESET_ALL + "\n")
    # statistics at the end
    if game_count != 0:
        print("Toatal score: %s/%s\n" % (won_count, game_count) )
    sys.exit(0)
   

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…


Game  1  Opponent: Rock     You: Scissor  	R:0.165565, P:0.046440, S:0.787995	You lose
Game  2  Opponent: Rock     You: Paper    	R:0.301882, P:0.680867, S:0.017251	You win!!
Game  3  Opponent: Paper    You: Paper    	R:0.007267, P:0.992441, S:0.000292	Draw
Game  4  Opponent: Paper    You: Paper    	R:0.381453, P:0.596011, S:0.022536	Draw
Game  5  Opponent: Rock     You: Rock     	R:0.668864, P:0.323671, S:0.007465	Draw
Game  6  Opponent: Rock     You: Paper    	R:0.251373, P:0.734951, S:0.013676	You win!!
Game  7  Opponent: Paper    You: Rock     	R:0.548067, P:0.409983, S:0.041950	You lose
Game  8: GO!
Toatal score: 2/8



SystemExit: 0

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
